In [2]:
import tensorflow as tf
import numpy as np
import os

In [3]:
feature_dimension = 90
frames = 100

x = tf.placeholder(shape = [None,feature_dimension,frames], dtype = tf.float32)
y = tf.placeholder(shape = [None,2], dtype = tf.float32)


In [4]:
#reshaping the input
input_x = tf.reshape(x,[-1,feature_dimension,frames,1])
conv1 = tf.layers.conv2d(input_x,
                        filters = 32,
                        kernel_size = 9,
                        activation = tf.nn.relu,
                        padding = 'VALID',
                        name = 'conv1'
                        )

conv2 = tf.layers.conv2d(conv1,
                        filters = 32,
                        kernel_size = 9,
                        activation = tf.nn.relu,
                        padding = 'VALID',
                        name = 'conv2'
                        )

conv_out = tf.layers.conv2d(conv2,
                        filters = 32,
                        kernel_size = 9,
                        activation = tf.nn.relu,
                        padding = 'VALID',
                        name = 'conv_out'
                        )

eplison = 1e-7

In [5]:
conv_out

<tf.Tensor 'conv_out/Relu:0' shape=(?, 66, 76, 32) dtype=float32>

In [6]:
def squash(vector,axis = -2, keep_dims = True):
    squared_norm = tf.reduce_sum(tf.square(vector), axis = axis, keep_dims = keep_dims)
    squash_factor = squared_norm / (1. + squared_norm)
    safe_norm = tf.sqrt(squared_norm + eplison)
    unit_vector = vector / safe_norm
    return squash_factor * unit_vector
    

In [7]:
#primary capsules

vector_dimension = 8
total_capsules = int(conv_out.shape[1]*conv_out.shape[2]*conv_out.shape[3] // vector_dimension )
raw_capsules = tf.reshape(conv_out ,shape = [-1, total_capsules , vector_dimension])
primary_capsules = squash(raw_capsules,-1,True)

In [8]:
print(primary_capsules)
batch_size = tf.shape(x)[0]

Tensor("mul:0", shape=(?, 20064, 8), dtype=float32)


In [9]:
selected_capsules_number = 512

selected_capsules_weights = tf.Variable( tf.random_normal(shape = [1,total_capsules,selected_capsules_number],
                                        stddev = 0.1,
                                       dtype = tf.float32))
print(selected_capsules_weights)

selected_capsules_weights_batches = tf.tile(selected_capsules_weights, [batch_size, 1, 1])
print(selected_capsules_weights_batches)
print(tf.transpose(primary_capsules))
selected_capsules = tf.matmul(primary_capsules,selected_capsules_weights_batches, transpose_a = True)

selected_capsules = tf.reshape(selected_capsules , shape = [-1,selected_capsules_number, vector_dimension])
print(selected_capsules)

<tf.Variable 'Variable:0' shape=(1, 20064, 512) dtype=float32_ref>
Tensor("Tile:0", shape=(?, 20064, 512), dtype=float32)
Tensor("transpose:0", shape=(8, 20064, ?), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 512, 8), dtype=float32)


In [10]:
output_vector = 16
output_labels = 2                                                     # [?,1152,2,16,8]

digit_capsule_weights = tf.Variable( tf.random_normal(shape = [1, selected_capsules_number, output_labels, output_vector, vector_dimension ],
                                                     stddev = 0.1,
                                                     dtype = tf.float32),
                                                       name = 'digit_capsule_weights')
print(digit_capsule_weights)
batch_size = tf.shape(x)[0]   #knows the shape of the tensor at the run time
digit_capsule_weights_tiled = tf.tile(digit_capsule_weights,[batch_size, 1, 1, 1, 1], name = 'digit_capsule_weights_tiled')

print(digit_capsule_weights_tiled)

<tf.Variable 'digit_capsule_weights:0' shape=(1, 512, 2, 16, 8) dtype=float32_ref>
Tensor("digit_capsule_weights_tiled:0", shape=(?, 512, 2, 16, 8), dtype=float32)


In [11]:
                                                                #

selected_caps_reshape = tf.reshape(selected_capsules, shape = [batch_size, selected_capsules_number, 1, vector_dimension, 1],
                                  name = 'selected_caps_reshape')
selected_caps_reshape_tiled = tf.tile(selected_caps_reshape,
                                          [1, 1, output_labels, 1 , 1 ],
                                         name = 'selected_caps_reshape_tiled')
print(selected_caps_reshape_tiled)

Tensor("selected_caps_reshape_tiled:0", shape=(?, 512, 2, 8, 1), dtype=float32)


In [12]:
all_capsules_pred_output = tf.matmul(digit_capsule_weights_tiled,selected_caps_reshape_tiled ,
                                    name = 'raw_predictions')
print(all_capsules_pred_output)

Tensor("raw_predictions:0", shape=(?, 512, 2, 16, 1), dtype=float32)


In [13]:
# dymanic routing 1

raw_weights = tf.zeros([batch_size, selected_capsules_number, output_labels, 1 , 1 ],
                          dtype = tf.float32,
                          name = 'raw_weights')
routing_weights = tf.nn.softmax(raw_weights,
                               dim = 2,
                                name = 'routing_weights')
weighted_predictions = tf.multiply(routing_weights, all_capsules_pred_output,
                                  name = 'weighted_predictions')
print(weighted_predictions)


Tensor("weighted_predictions:0", shape=(?, 512, 2, 16, 1), dtype=float32)


In [14]:
# sum over all 512 capsules

weighted_sum = tf.reduce_sum(weighted_predictions, 
                            axis = 1, keep_dims = True,
                            name = 'weighted_sum')
pred_vector_r1 = squash(weighted_sum)

pred_vector_r1

<tf.Tensor 'mul_1:0' shape=(?, 1, 2, 16, 1) dtype=float32>

In [15]:
# agreements
pred_vector_r1_tiled = tf.tile(pred_vector_r1,
                              [1,selected_capsules_number, 1, 1, 1])

print(tf.transpose(all_capsules_pred_output))
print(pred_vector_r1_tiled)
agreements = tf.matmul(all_capsules_pred_output,pred_vector_r1_tiled, 
                     transpose_a = True, name = 'agreements')
print(agreements)

Tensor("transpose_2:0", shape=(1, 16, 2, 512, ?), dtype=float32)
Tensor("Tile_1:0", shape=(?, 512, 2, 16, 1), dtype=float32)
Tensor("agreements:0", shape=(?, 512, 2, 1, 1), dtype=float32)


In [16]:
# round 2

raw_weights = raw_weights + agreements
routing_weights_r2 = tf.nn.softmax(raw_weights,
                               dim = 2,
                                name = 'routing_weights_r2')
weighted_predictions_r2 = tf.multiply(routing_weights_r2, all_capsules_pred_output,
                                  name = 'weighted_predictions_r2')
weighted_sum_r2 = tf.reduce_sum(weighted_predictions_r2, 
                            axis = 1, keep_dims = True,
                            name = 'weighted_sum_r2')
pred_vector_r2 = squash(weighted_sum_r2)

print(pred_vector_r2)
epsilon = 1e-7
predicted_vector = pred_vector_r2





sum_squares = tf.reduce_sum( tf.square(predicted_vector), axis = -2, keep_dims = False, name = 'sum_squares')
safe_norm = tf.sqrt(sum_squares + epsilon)
print(safe_norm)

pred_prob = tf.reshape(safe_norm, shape =  [batch_size ,2])
pred_labels = tf.argmax(pred_prob, axis = 1)
print(pred_labels)


# loss

m_plus = 0.9
m_minus = 0.1
lamda =  - 0.5

present_error_raw = tf.square(tf.maximum(0.0, m_plus - safe_norm))
present_error = tf.reshape(present_error_raw, shape = [-1, 2])
print(present_error)

absent_error_raw = tf.square(tf.maximum(0.0, safe_norm - m_minus))
absent_error = tf.reshape(absent_error_raw, shape = [-1, 2])
print(absent_error)

T = y

L = tf.add(T * present_error , lamda * ((1.0 - T) * absent_error), name = 'L')

margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis = 1), name = 'margin_loss')

optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(margin_loss, name = 'train_step')

Tensor("mul_2:0", shape=(?, 1, 2, 16, 1), dtype=float32)
Tensor("Sqrt_3:0", shape=(?, 1, 2, 1), dtype=float32)
Tensor("ArgMax:0", shape=(?,), dtype=int64)
Tensor("Reshape_8:0", shape=(?, 2), dtype=float32)
Tensor("Reshape_9:0", shape=(?, 2), dtype=float32)


In [ ]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [ ]:

from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    indexs = (len(list(os.listdir(test_path))))//2
    
    for folders in range(1, indexs+1):
        features = np.load(os.path.join(test_path, "features_"+str(folders)+".npy"))
        labels = np.load(os.path.join(test_path, "labels_"+str(folders)+".npy"))
        patches = labels.shape[0]
        patch_prob = session.run(pred_prob, feed_dict = {x:features})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
        
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [ ]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config = config)

session = tf.InteractiveSession()

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep = None)

features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\overlapData_90D'
development_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\devoverlapData_90D'
eval_features_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evalcontextData_90D'
model_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Models_90D_caps_net_v2_3c_1fc_512'

dev_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Dev_sp_90D_caps_net_v2_3c_1fc_512'
eval_sp_path = 'C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Eval_sp_90D_caps_net_v2_3c_1fc_512'


In [ ]:
#initialize session

session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)

batches = (len(list(os.listdir(features_path))))//2

for i in range(0,10):
    for j in range(1,batches+1):
        batch_name = 'batch_'+str(j)+'.npy'
        label_name = 'label_'+str(j)+'.npy'
        
        features = np.load(os.path.join(features_path,batch_name))
        labels = np.load(os.path.join(features_path,label_name))
        
        train_step.run(session = session, feed_dict = {x : features, y: labels})
    save_path = saver.Save(session, os.path.join(model_path,'Model_'+str(i+1)))
    
    print("iteration   ", (i+1))
    print()
        